In [1]:
import pickle
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib

import sys
sys.path.insert(0, '../../')
import LKBMachine as dl
import LossLKB as lf
import Data as dat

%matplotlib inline

# Setup

In [ ]:
# Figure parameters
plot_save_path = './analysis_results/'
font = {'family': 'DejaVu Sans', 'size': 18}
matplotlib.rc('font', **font)
fontsize = 18
figsize = (15, 10)
dpisave = 300

# Initialize the compute device
DEVICE = '/GPU:0'
GPUS = tf.config.experimental.list_physical_devices('GPU')
if GPUS:
    try:
        for gpu in GPUS:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
else:
    DEVICE = '/CPU:0'

tf.keras.backend.set_floatx('float64')  # !! Set precision for the entire model here
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))
print("Num GPUs available: {}".format(len(GPUS)))
print("Training at precision: {}".format(tf.keras.backend.floatx()))
print("Training on device: {}".format(DEVICE))

# Load model and data

In [ ]:
# SET THIS PATH (w/o file extension!). Both '.pkl' and '.h5' files should have same name
model_path = './trained_models/pendulum_2021-05-25-0954/epoch_100_loss_0.057'
model_hyp_params = model_path + '.pkl'
model_weights = model_path + '.h5'

# Load the hyper parameters
hyp_params = pickle.load(open(model_hyp_params, 'rb'))

# Set Tensorflow backend precision
tf.keras.backend.set_floatx(hyp_params['precision'])
print("Using precision: {}\n".format(tf.keras.backend.floatx()))

# Generate test data
hyp_params['num_init_conds'] = 20
hyp_params['time_final'] = 10
hyp_params['num_time_steps'] = int(hyp_params['time_final'] / hyp_params['delta_t'])
from scipy.integrate import solve_ivp
def vdp(t, x):
    return [x[1], mu * (1 - x[0] ** 2) * x[1] - x[0]]
mu = hyp_params['mu']
icx = np.random.uniform(-2, 2, hyp_params['num_init_conds'])
icy = np.random.uniform(-2, 2, hyp_params['num_init_conds'])
tspan = np.array([0, hyp_params['time_final']])
dts = np.arange(0, hyp_params['time_final'], hyp_params['delta_t'])
X = np.zeros(shape=(hyp_params['num_init_conds'], 2, hyp_params['num_time_steps']))
for ii, ic in enumerate(zip(icx, icy)):
    tmp = solve_ivp(vdp, t_span=tspan, y0=ic, method='RK45', t_eval=dts)
    X[ii, :, :] = tmp.y
test_data = tf.transpose(X, perm=[0, 2, 1])
test_data = tf.cast(test_data, dtype=hyp_params['precision'])
print("Test data shape: {}".format(test_data.shape))

In [4]:
# Fix hyper parameters for running the model on test data
hyp_params['pretrain'] = False
hyp_params['batch_size'] = test_data.shape[0]
hyp_params['num_time_steps'] = test_data.shape[1]
hyp_params['latent_dim'] = test_data.shape[2]
hyp_params['phys_dim'] = test_data.shape[2]

# Load the trained DLDMD model weights
model = dl.LKBMachine(hyp_params)
model.num_pred_steps = hyp_params['num_time_steps']
model.time_final = int(hyp_params['num_time_steps'] * model.delta_t)
model(test_data)
model.load_weights(model_weights)

# Initialize the loss function
loss = lf.LossLKB(hyp_params)
print("Number of prediction steps: ", model.num_pred_steps)

Using precision: float64

Test data shape: (10, 1500, 2)


# Run the trained model

In [ ]:
with tf.device(DEVICE):
    preds = model(test_data, training=False)
    losses = loss(preds, test_data)

[y, x_ae, x_adv, y_adv, weights, evals_real, evals_cmplx] = preds
print("Loss: {loss:2.7f}".format(loss=losses.numpy()))
print("Log10 Loss: {loss:2.7f}".format(loss=np.log10(losses.numpy())))

# Visualize results

In [ ]:
fig = plt.figure(3141, figsize=(25,10))
ax1 = plt.subplot(1, 1, 1)
for ii in range(test_data.shape[0]):
    ax1.plot(test_data[ii, :, 0], test_data[ii, :, 1], 'r-', lw=0.5)
    ax1.plot(x_adv[ii, :, 0], x_adv[ii, :, 1], 'b.', ms=0.5)
ax1.plot(x_adv[:, 0, 0], x_adv[:, 0, 1], 'go', label='initial condition')
ax1.plot(x_adv[0, 0, 0], x_adv[0, 0, 1], 'b.', label='LKBMachine')
ax1.plot(test_data[0, :, 0], test_data[0, :, 1], 'r-', lw=0.5, label='test data')
ax1.legend()
ax1.axis('equal')
plt.suptitle("LKBMachine predictions of nonlinear pendulum phase orbits")
plt.show()

In [7]:
fig = plt.figure(1, figsize=(20,20))
ax1 = plt.subplot(2, 2, 1)
ax2 = plt.subplot(2, 2, 2)
ax3 = plt.subplot(2, 2, 3)
ax4 = plt.subplot(2, 2, 4)
for ii in range(test_data.shape[0]):
    ax1.plot(test_data[ii, :, 0], test_data[ii, :, 1], '-')
    ax2.plot(x_adv[ii, :, 0], x_adv[ii, :, 1], '-')
    ax3.plot(y_adv[ii, :, 0], y_adv[ii, :, 1], '-')
    ax4.plot(x_ae[ii, :, 0], x_ae[ii, :, 1], '-')
ax1.plot(test_data[:, 0, 0], test_data[:, 0, 1], '.')
ax2.plot(x_adv[:, 0, 0], x_adv[:, 0, 1], '.')
ax3.plot(y_adv[:, 0, 0], y_adv[:, 0, 1], '.')
ax4.plot(x_ae[:, 0, 0], x_ae[:, 0, 1], '.')
ax1.axis('equal')
ax1.set_xlim([-3.1, 3.1])
ax1.set_ylim([-3.0, 3.0])
ax1.set_xlabel("$x_1$", fontsize=fontsize)
ax1.set_ylabel("$x_2$", fontsize=fontsize)
ax1.set_title("Test Data (x)", fontsize=fontsize)
ax2.axis('equal')
ax2.set_xlim([-3.1, 3.1])
ax2.set_ylim([-3.0, 3.0])
ax2.set_xlabel("$x_1$", fontsize=fontsize)
ax2.set_ylabel("$x_2$", fontsize=fontsize)
ax2.set_title("Encoded-Advanced-Decoded (x_adv))", fontsize=fontsize)
ax3.axis('equal')
ax3.set_xlabel("$y_1$", fontsize=fontsize)
ax3.set_ylabel("$y_2$", fontsize=fontsize)
ax3.set_title("Encoded-Advanced (y_adv))", fontsize=fontsize)
ax4.axis('equal')
ax4.set_xlim([-3.1, 3.1])
ax4.set_ylim([-3.0, 3.0])
ax4.set_xlabel("$x_1$", fontsize=fontsize)
ax4.set_ylabel("$x_2$", fontsize=fontsize)
ax4.set_title("Encoded-Decoded (x_ae)", fontsize=fontsize)

plt.show()